In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import os
import pickle

C:\Users\surya\AppData\Local\Temp\ipykernel_16556\3316206840.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_data = pd.read_csv(r'C:\Users\surya\OneDrive\Desktop\SlashMark\Fake_News_Detection-master\Fake_News_Detection-master\train.csv')
test_data = pd.read_csv(r'C:\Users\surya\OneDrive\Desktop\SlashMark\Fake_News_Detection-master\Fake_News_Detection-master\test.csv')
val_data = pd.read_csv(r'C:\Users\surya\OneDrive\Desktop\SlashMark\Fake_News_Detection-master\Fake_News_Detection-master\valid.csv')

In [3]:
print(train_data)

                                               Statement  Label
0      Says the Annies List political group supports ...  False
1      When did the decline of coal start? It started...   True
2      Hillary Clinton agrees with John McCain "by vo...   True
3      Health care reform legislation is likely to ma...  False
4      The economic turnaround started at the end of ...   True
...                                                  ...    ...
10235  There are a larger number of shark attacks in ...   True
10236  Democrats have now become the party of the [At...   True
10237  Says an alternative to Social Security that op...   True
10238  On lifting the U.S. Cuban embargo and allowing...  False
10239  The Department of Veterans Affairs has a manua...  False

[10240 rows x 2 columns]


In [4]:
print(test_data)

                                              Statement  Label
0     Building a wall on the U.S.-Mexico border will...   True
1     Wisconsin is on pace to double the number of l...  False
2     Says John McCain has done nothing to help the ...  False
3     Suzanne Bonamici supports a plan that will cut...   True
4     When asked by a reporter whether hes at the ce...  False
...                                                 ...    ...
2546  For the first time in more than a decade, impo...   True
2547  Says Donald Trump has bankrupted his companies...   True
2548  John McCain and George Bush have "absolutely n...   True
2549  A new poll shows 62 percent support the presid...  False
2550  No one claims the report vindicating New Jerse...  False

[2551 rows x 2 columns]


In [5]:
print(val_data)

                                              Statement  Label
0     We have less Americans working now than in the...  FALSE
1     When Obama was sworn into office, he DID NOT u...  FALSE
2     Says Having organizations parading as being so...  FALSE
3        Says nearly half of Oregons children are poor.   TRUE
4     On attacks by Republicans that various program...   TRUE
...                                                 ...    ...
2566  Says Donald Trump has bankrupted his companies...   TRUE
2567  John McCain and George Bush have "absolutely n...   TRUE
2568  A new poll shows 62 percent support the presid...  FALSE
2569  No one claims the report vindicating New Jerse...  FALSE
2570   >>>>>>> f88339ab0722caf66aff4094544f81ce23820d3b    NaN

[2571 rows x 2 columns]


In [6]:
# Define function to convert True/False to 1/0
def convert_to_binary(value):
    if isinstance(value, bool):
        return 1 if value else 0
    elif isinstance(value, str):
        return 1 if "TRUE" in value.upper() else 0
    elif pd.isna(value):  # Handle NaN values
        return 0  # You can choose to assign NaN values to 0 or handle them differently based on your requirements
    else:
        return 0  # Handle unexpected values, such as float or other types

# Converting target column to binary

train_data['Label'] = train_data['Label'].apply(convert_to_binary)
test_data['Label'] = test_data['Label'].apply(convert_to_binary)
val_data['Label'] = val_data['Label'].apply(convert_to_binary)


In [7]:
print(train_data)

                                               Statement  Label
0      Says the Annies List political group supports ...      0
1      When did the decline of coal start? It started...      1
2      Hillary Clinton agrees with John McCain "by vo...      1
3      Health care reform legislation is likely to ma...      0
4      The economic turnaround started at the end of ...      1
...                                                  ...    ...
10235  There are a larger number of shark attacks in ...      1
10236  Democrats have now become the party of the [At...      1
10237  Says an alternative to Social Security that op...      1
10238  On lifting the U.S. Cuban embargo and allowing...      0
10239  The Department of Veterans Affairs has a manua...      0

[10240 rows x 2 columns]


In [8]:
print(test_data)

                                              Statement  Label
0     Building a wall on the U.S.-Mexico border will...      1
1     Wisconsin is on pace to double the number of l...      0
2     Says John McCain has done nothing to help the ...      0
3     Suzanne Bonamici supports a plan that will cut...      1
4     When asked by a reporter whether hes at the ce...      0
...                                                 ...    ...
2546  For the first time in more than a decade, impo...      1
2547  Says Donald Trump has bankrupted his companies...      1
2548  John McCain and George Bush have "absolutely n...      1
2549  A new poll shows 62 percent support the presid...      0
2550  No one claims the report vindicating New Jerse...      0

[2551 rows x 2 columns]


In [9]:
train_data.describe()

,Label
count,10240.000000
mean,0.561719
std,0.496200
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Statement  10240 non-null  object
 1   Label      10240 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 160.1+ KB


In [11]:
p_s=PorterStemmer()
p_s

<PorterStemmer>

In [12]:
p_s.stem("He is observing everything and the news was reported early! in the morning itself")

'he is observing everything and the news was reported early! in the morning itself'

In [13]:
def stemming(content):
    con=re.sub('[^a-z,A-z]','  ',content)
    con=con.lower()
    con=con.split()
    con=[p_s.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

print(stemming('Hi my name is satwik and reported early before!'))

hi name satwik report earli


In [14]:
train_data['Statement']=train_data['Statement'].apply(stemming)

In [15]:
test_data['Statement']=test_data['Statement'].apply(stemming)
val_data['Statement']=val_data['Statement'].apply(stemming)

In [16]:
x=train_data["Statement"]
y=train_data["Label"]

In [17]:
from sklearn.model_selection import train_test_split


In [18]:
# Separate features and target variable
x_train=train_data["Statement"]
x_test=test_data["Statement"]

y_train=train_data["Label"]
y_test=test_data["Label"]

In [19]:
x_train

0        say anni list polit group support third trimes...
1        declin coal start start natur ga took start be...
2        hillari clinton agre john mccain vote give geo...
3        health care reform legisl like mandat free sex...
4                         econom turnaround start end term
                               ...                        
10235    larger number shark attack florida case voter ...
10236      democrat becom parti [atlanta] metro area black
10237    say altern social secur oper galveston county,...
10238               lift u cuban embargo allow travel cuba
10239    depart veteran affair manual tell veteran stuf...
Name: Statement, Length: 10240, dtype: object

In [20]:
y_train

0        0
1        1
2        1
3        0
4        1
        ..
10235    1
10236    1
10237    1
10238    0
10239    0
Name: Label, Length: 10240, dtype: int64

In [21]:
import pandas as pd

# Convert y_train to a pandas Series (assuming y_train is a numpy array or pandas DataFrame)
y_train_series = pd.Series(y_train)

# Count the occurrences of each class
class_counts = y_train_series.value_counts()

# Print the count of occurrences of each class
print("Class 0 count:", class_counts[0])
print("Class 1 count:", class_counts[1])

Class 0 count: 4488
Class 1 count: 5752


In [22]:
x_test

0              build wall u mexico border take liter year
1                 wisconsin pace doubl number layoff year
2                      say john mccain done noth help vet
3       suzann bonamici support plan cut choic medicar...
4       ask report whether he center crimin scheme vio...
                              ...                        
2546    first time decade, import account less half oi...
2547    say donald trump bankrupt compani once, twice ...
2548    john mccain georg bush absolut plan univers he...
2549    new poll show percent support presid plan refo...
2550    one claim report vindic new jersey gov chri ch...
Name: Statement, Length: 2551, dtype: object

In [23]:
y_test

0       1
1       0
2       0
3       1
4       0
       ..
2546    1
2547    1
2548    1
2549    0
2550    0
Name: Label, Length: 2551, dtype: int64

In [24]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (10240,)
y_train shape: (10240,)
x_test shape: (2551,)
y_test shape: (2551,)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer()

In [26]:
vect.fit(x_train)
x_train=vect.transform(x_train)
x_test=vect.transform(x_test)

In [27]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()

In [28]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [29]:
model.score(x_test,y_test)

0.551156409251274

In [30]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Define the XGBoost model
model = xgb.XGBClassifier(
    objective='binary:logistic',  # Use binary classification
    eval_metric='logloss'  # Evaluation metric to monitor during training
)

# Train the model
model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5985887887103096


In [31]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Define the XGBoost model
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

# Define hyperparameters to tune and their distributions for Randomized Search
param_dist = {
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Perform Randomized Search Cross Validation
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   scoring='accuracy', n_iter=10, cv=3, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Use the best model from Randomized Search
best_model = random_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Best Parameters: {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
Best Score: 0.5875974576685752
Accuracy: 0.5833006664053313


In [32]:
from sklearn.metrics import classification_report, roc_auc_score

# Make predictions on the test set using the best model
y_pred = best_model.predict(x_test)



# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.25      0.35      1169
           1       0.58      0.87      0.69      1382

    accuracy                           0.58      2551
   macro avg       0.59      0.56      0.52      2551
weighted avg       0.59      0.58      0.54      2551



In [33]:
pickle.dump(vect , open('vector.pkl','wb'))
pickle.dump(model , open('model.pkl','wb'))

In [34]:
pickle.load(open('vector.pkl','rb'))

TfidfVectorizer()